In [5]:
import json
import spacy
from spacy import displacy

with open("/Users/leo/PycharmProjects/EECS4080/train-v2.0.json") as f:
    data = json.load(f)
count = 0
result = []
for i in range(2):
    for d in data["data"][i]["paragraphs"]:
        if count == 20:
            break
        if "context" in d.keys():
            print(d["context"])
            result.append(d["context"])
            print("\n---------------------------------------------------------------------------\n")
            count += 1
    
print(count)
            

Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".

---------------------------------------------------------------------------

Following the disbandment of Destiny's Child in June 2005, she released her second solo album, B'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golde

In [8]:
nlp = spacy.load("en_core_web_lg")
text="\n---------------------------------------------------------------------------------\n".join(result)
print(text)   
doc = nlp(text)
displacy.serve(doc, style="ent")


ValueError: Cannot create vectors table with dimension 0.
If you're using pre-trained vectors, are the vectors loaded?

##### Obtain top 10 frequently occurred entities recognized as person
Generally, NER of the full name of a person is more accurate than the NER of a given name \
Based on this, I do the following steps: 

1. Traverse the entity list of the document, if the entity is labeled as a person, we obtain the person's full name (change to lowercase, remove stop words, remove punctuation). We add the full name to a dictionary and set the count to 1. (full name is key, count is value). For each entity, when added to the dictionary, we check if it is a substring of the keys of the dict. If so, it is a given name, we increment the count of the full name, and not add the given name to the dictionary. If it is not a substring of the keys, then it is a new name, we add it to the dictionary and set count to 1.

2. Convert the dictionary to a list, and sort it in reverse order so we can have the top10 most frequently occurred person's name.

3. Remove entities in the top10 person's name from the entity list of the document.

4. Update the label of the names to PERSON, and added them back to the entity list. 

The reason for doing this is that, we can make sure the given name and the full name are labeled consistently with PERSON. Another reason is that NER of full name is more accurate, so we want to trust full name entity labels rather than those of Given names.  



In [ ]:
import string
from spacy.tokens import Span

text = "\n\n".join(result)
doc = nlp(text)
dict = {}


# for chunk in doc.noun_chunks:
#     print(f"{chunk.text:50}{chunk.root}")
    
    
for ent in doc.ents:
    entity = ""
    entity_type = ""
    for token in ent:
        # print(token.text, token.ent_type_, type(token.ent_type_), "\n")
        entity_type = token.ent_type_        
        if entity_type != "PERSON":
            continue
        if token.is_stop:  # remove stop words
            continue
        if token.text in string.punctuation:
            continue
        token = token.lemma_.lower()  # convert token to lemma and lowercase
        if token.isnumeric():  # remove numbers
            continue
        entity += token + " "
    entity = entity.strip()
    found = False
    for k in dict.keys():
        if entity in k.split():
            dict[k] += 1
            found = True
    if not found:
        dict[entity] = 1


dict_list = []
for key, value in dict.items():
    dict_list.append((key, value))

# sort the entity -> count dict to get top10 person entities        
dict_list = sorted(dict_list, key=lambda x: x[1], reverse=True)
top10 = dict_list[:10]


top10_list = []
for e in top10:
    top10_list.append(e[0])
    
    
top10_str = " ".join(top10_list)
print(top10_list)

In [ ]:
change_list = []
change_set = set()
for ent in doc.ents:
    if ent.text.lower() in top10_str and ent.label_ != "PERSON":
        change_list.append(ent)
        change_set.add(ent.text)
        print(ent.vector)
        ents.remove(ent)
        doc.ents = tuple(ents)
update_ent = []
print(change_set)
print()
for e in change_list:
    span = Span(doc, e.start, e.end, label="PERSON")
    update_ent.append(span)
print(update_ent)
doc.ents = list(doc.ents) + update_ent


In [ ]:
displacy.serve(doc, style="ent")

Improve entity recognition with similarity

In [ ]:
doc1 = nlp(text)

# add all the ent strings into a set
ent_set = set()
# remove repeat ent and ent that are not noun
# add all the ent strings that are noun into a dict as keys, ent as values
ent_Noun_dict = {}
for ent in doc1:
    ent_set.add(ent.text)
    if ent.pos_ == "NOUN":
        if ent.text not in ent_Noun_dict:
            ent_Noun_dict[ent.text] = ent

# add ent pairs to similarity_dict
# calculate the similarity between each ent pairs
similarity_dict = {}
for k1 in ent_Noun_dict.keys():
    for k2 in ent_Noun_dict.keys():
        if k1 not in similarity_dict:
            similarity_dict[k1] = [(ent_Noun_dict[k2], ent_Noun_dict[k1].similarity(ent_Noun_dict[k2]))]
        else:
            similarity_dict[k1].append((ent_Noun_dict[k2], ent_Noun_dict[k1].similarity(ent_Noun_dict[k2])))
print("done")
